In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

c:\Users\user\Documents\Time_series\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [4]:
url = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/main/data/wiki_machine_learning.csv"
df = pd.read_csv(url, sep=" ")

# Оставляем только ненулевые просмотры и нужные колонки
df = df[df["count"] != 0]
df['date'] = pd.to_datetime(df['date'])
df = df[['date', 'count']]
df.columns = ['ds', 'y']  # переименовываем для Prophet
df.head(10)

,ds,y
81,2015-01-01,1414
80,2015-01-02,1920
79,2015-01-03,1338
78,2015-01-04,1404
77,2015-01-05,2264
76,2015-01-06,2327
75,2015-01-07,2469
74,2015-01-08,2349
73,2015-01-09,2279
63,2015-01-10,1600


Часть 1: Прогноз с помощью Facebook Prophet

In [ ]:
# Часть 1: Прогноз с помощью Facebook Prophet
predictions = 30
train_df = df[:-predictions].copy()
test_df = df[-predictions:].copy()

# 1. Создаём и обучаем модель
model = Prophet()
model.fit(train_df)

# 2. Создаём прогноз на 30 дней
future = model.make_future_dataframe(periods=predictions)
forecast = model.predict(future)

# 3. Определяем дату для прогноза на 20 января
# Данные заканчиваются в 2016 году, значит 20 января 2016
last_train_date = train_df['ds'].max()
print(f"Последняя дата в обучающих данных: {last_train_date}")

# Вычисляем 20 января 2016 года
 
 
jan_20_2016 = pd.Timestamp('2016-01-20')

# Находим этот день в прогнозе
jan_20_forecast = forecast[forecast['ds'] == jan_20_2016]

if len(jan_20_forecast) > 0:
    prediction_value = jan_20_forecast['yhat'].values[0]
    print(f"\nВопрос 1: Прогноз на 20 января 2016 года:")
    print(f"Предсказание: {prediction_value:.0f} просмотров")
else:
    # Если не нашли, берем 20-й день прогноза
    prediction_20th_day = forecast.iloc[-predictions + 19]['yhat']  # 20-й день из 30
    print(f"\nВопрос 1: Прогноз на 20-й день (20 января 2016):")
    print(f"Предсказание: {prediction_20th_day:.0f} просмотров")

# 4. Оцениваем качество на последних 30 днях
forecast_test = forecast.iloc[-predictions:]

mae = mean_absolute_error(test_df['y'], forecast_test['yhat'])
mape = mean_absolute_percentage_error(test_df['y'], forecast_test['yhat'])

print(f"\nВопрос 2: MAPE = {mape:.4f} (или {mape*100:.2f}%)")
print(f"Вопрос 3: MAE = {mae:.0f} просмотров")

15:18:59 - cmdstanpy - INFO - Chain [1] start processing
15:18:59 - cmdstanpy - INFO - Chain [1] done processing


Последняя дата в обучающих данных: 2015-12-21 00:00:00

Вопрос 1: Прогноз на 20 января 2016 года:
Предсказание: 3422 просмотров

Вопрос 2: MAPE = 0.3435 (или 34.35%)
Вопрос 3: MAE = 597 просмотров


Часть 2: Прогноз с помощью ARIMA (SARIMAX)

In [6]:
# Вопрос 4: Проверка стационарности ряда с помощью критерия Дики-Фуллера (ADF)
result = adfuller(df['y'])  # Проводим тест на исходном ряде 'y' (просмотры)

print("Вопрос 4: Проверка стационарности ряда (критерий Дики-Фуллера):")
print(f"ADF Statistic: {result[0]:.4f}")
print(f"p-value: {result[1]:.4f}")
print("Критические значения:")
for key, value in result[4].items():
    print(f"   {key}: {value:.4f}")

# Интерпретация результата теста
if result[1] <= 0.05:
    print("Вывод: p-value ≤ 0.05 -> ряд СТАЦИОНАРЕН.")
else:
    print("Вывод: p-value > 0.05 -> ряд НЕСТАЦИОНАРЕН.")

Вопрос 4: Проверка стационарности ряда (критерий Дики-Фуллера):
ADF Statistic: -3.2889
p-value: 0.0154
Критические значения:
   1%: -3.4483
   5%: -2.8694
   10%: -2.5710
Вывод: p-value ≤ 0.05 -> ряд СТАЦИОНАРЕН.


 Подбор лучших параметров модели SARIMAX по критерию AIC

In [ ]:
# Для ускорения подбора будем перебирать небольшие значения параметров.
# В реальных задачах сетка поиска может быть больше.

# Параметры для перебора: p, d, q (несезонные) и P, D, Q, S (сезонные)
# Здесь S=7 (недельная сезонность, так как данные ежедневные)
p_values = range(0, 2)  # авторегрессия
d_values = range(0, 2)  # разность
q_values = range(0, 2)  # скользящее среднее
seasonal_pdq = [(1, 1, 1, 7)]  # попробуем один вариант сезонных параметров

best_aic = float("inf")  # устанавливаем начальное значение AIC как бесконечность
best_params = None

# Перебираем параметры
for p in p_values:
    for d in d_values:
        for q in q_values:
            for P, D, Q, S in seasonal_pdq:
                try:
                    # Создаём и обучаем модель SARIMAX
                    model = sm.tsa.statespace.SARIMAX(
                        df['y'],  # данные (просмотры)
                        order=(p, d, q),  # несезонные параметры
                        seasonal_order=(P, D, Q, S),  # сезонные параметры
                        enforce_stationarity=False,
                        enforce_invertibility=False
                    )
                    results = model.fit(disp=False)  # обучаем, disp=False убирает лишний вывод
                    
                    # Сравниваем AIC текущей модели с лучшим найденным
                    if results.aic < best_aic:
                        best_aic = results.aic
                        best_params = (p, d, q, P, D, Q, S)
                except:
                    continue  # если параметры не подходят, переходим к следующим

 
print(f"Лучшие параметры (p,d,q,P,D,Q,S): {best_params}")
print(f"Лучший AIC: {best_aic:.2f}")

c:\Users\user\Documents\Time_series\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\user\Documents\Time_series\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\user\Documents\Time_series\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\user\Documents\Time_series\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: Va

Вопрос 5: Лучшая модель SARIMAX по критерию AIC:
Лучшие параметры (p,d,q,P,D,Q,S): (1, 1, 1, 1, 1, 1, 7)
Лучший AIC: 5351.68


Выводы

Модель предсказывает 3422 просмотра на 20 января 2016

Модель уловила общий тренд (скорее всего, рост просмотров к концу года) и недельную сезонность (меньше просмотров в выходные). Но точность неидеальная

MAPE=34% → Ошибка довольно большая. Модель ошибается в среднем на 34% от реального значения. Например, если реально 1000 просмотров, прогноз может быть от 660 до 1340.

MAE=597 просмотров → В абсолютных числах ошибка около 600 просмотров.

Лучшая SARIMAX модель (1,1,1,1,1,1,7)
AIC=5351.68 → чем меньше AIC, тем лучше

Prophet говорит о росте (прогноз 3422 — это выше среднего значения).

Дики-Фуллер говорит "стационарен" (нет тренда).

  Возможно, в данных есть сезонность без явного тренда. Например, просмотры растут к концу недели или месяца, но годового тренда нет.